---

📘 LightGBM 이론 요약

---

**LightGBM (Light Gradient Boosting Machine)**은 

- XGBoost의 속도와 메모리 문제를 개선하기 위해 개발된 부스팅 모델입니다. 
- 대규모 데이터셋에서 특히 강력한 성능을 발휘합니다.

---

**주요 특징**

- 리프 중심(Leaf-wise) 트리 성장: 손실을 가장 많이 줄일 수 있는 리프 노드를 지속적으로 분할하여 트리를 만듭니다.  

                               다른 모델보다 빠르게 학습할 수 있습니다.
                                
- 빠른 학습 속도: 리프 중심 성장 방식과 특성 분할을 위한 효율적인 기술 덕분에 XGBoost보다 훨씬 빠릅니다.
- 메모리 효율: 데이터를 미리 이산적인 '빈(bin)'으로 만들어 메모리 사용량을 줄입니다.

---


In [1]:
# LightGBM 모델 실습 코드

# 1. 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

print("LightGBM 모델 실습 시작...")

# 2. 데이터셋 로드
# scikit-learn에서 제공하는 붓꽃(Iris) 데이터셋 사용
iris = load_iris()
X, y = iris.data, iris.target

print(f"데이터셋 로드 완료. 특성(X)의 shape: {X.shape}, 레이블(y)의 shape: {y.shape}")

# 3. 학습 및 테스트 데이터 분리
# 전체 데이터의 80%를 학습용, 20%를 테스트용으로 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

# 4. LightGBM 모델 정의 및 파이프라인 생성
# LightGBM Classifier 모델 정의
# random_state: 재현성을 위한 시드값
lgbm_model = LGBMClassifier(random_state=42)

# 전처리와 모델 학습을 한 번에 처리하는 파이프라인 구축
# StandardScaler: 특성을 표준화하여 모델 학습 효율을 높임
# classifier: LightGBM 모델
pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                           ('classifier', lgbm_model)])

print("\n--- LightGBM 모델 학습 시작 ---")

# 5. 파이프라인을 사용하여 모델 학습
pipeline.fit(X_train, y_train)

print("--- 학습 완료 ---")

# 6. 테스트 데이터로 예측 및 정확도 평가
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nLightGBM 테스트 정확도: {accuracy:.4f}")

# 7. 교차 검증을 통한 모델 성능 평가
# cv=5: 5개의 폴드로 나누어 교차 검증 진행
cv_scores = cross_val_score(pipeline, X, y, cv=5)

print("\n--- 교차 검증 결과 ---")
print(f"각 폴드별 정확도: {cv_scores}")
print(f"교차 검증 평균 정확도: {cv_scores.mean():.4f} (표준편차: {cv_scores.std():.4f})")

print("\nLightGBM 모델 실습 완료.")


LightGBM 모델 실습 시작...
데이터셋 로드 완료. 특성(X)의 shape: (150, 4), 레이블(y)의 shape: (150,)
학습 데이터 크기: (120, 4)
테스트 데이터 크기: (30, 4)

--- LightGBM 모델 학습 시작 ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 87
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 4
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

---

### 📊 LightGBM 모델 실습 결과 분석

---

1.  **경고 메시지 (`No further splits with positive gain`)**
    * LightGBM 학습 과정에서 `No further splits with positive gain, best gain: -inf` 경고가 반복적으로 나타났습니다.
    * 붓꽃(Iris) 데이터셋이 매우 단순해서 모델이 더 이상 성능을 개선할 유의미한 분할 지점을 찾지 못했음을 의미합니다. 
    * 모델 학습에 문제가 있는 것이 아니라, 데이터 특성 때문에 나타나는 정상적인 현상입니다.
    * LightGBM 모델이 더 이상 성능을 크게 향상시킬 수 있는 분할 지점(split)을 찾지 못했다는 의미입니다. 
    * 즉, 모델이 현재 상태에서 최적의 학습 상태에 도달했거나, 데이터셋이 매우 단순하여 더 이상 분할할 필요가 없다고 판단한 것입니다.

---

2.  **테스트 정확도**
    * **LightGBM 테스트 정확도: `0.9667`**
    * 이는 모델이 `30개`의 테스트 데이터 중 약 `29개`(0.9667 * 30 ≈ 29)를 정확하게 예측했다는 의미입니다. 
    * XGBoost의 테스트 정확도 `0.9333`보다 약간 더 높게 나왔습니다.

---

3.  **교차 검증 결과**
    * **각 폴드별 정확도**: [`0.9666`, `0.9666`, `0.8666`, `0.9333`, `1.0000`]
    * **평균 정확도**: `0.9467`
    * **표준편차**: `0.0452`
    * LightGBM은 교차 검증에서 1.00의 정확도를 기록한 폴드도 있었지만, `0.8666`으로 낮은 정확도를 기록한 폴드도 있었습니다. 
    * 이 때문에 **표준편차(0.0452)**가 XGBoost(0.0163)보다 더 크게 나타났습니다.
    * 이는 LightGBM이 `리프 중심(Leaf-wise)` 성장 방식을 사용하기 때문에 상대적으로 과적합에 취약할 수 있다는 이론적 특징을 실습으로 확인한 결과라고 볼 수 있습니다.

---

### XGBoost와 LightGBM 결과 비교

| 구분 | **XGBoost** | **LightGBM** |
| :--- | :--- | :--- |
| **테스트 정확도** | 0.9333 | **0.9667** |
| **교차 검증 평균 정확도** | **0.9533** | 0.9467 |
| **교차 검증 표준편차** | **0.0163 (안정적)** | 0.0452 (불안정) |

- 이번 실습에서는 테스트 정확도만 보면 LightGBM이 더 높았지만, 

- 교차 검증의 평균 정확도와 표준편차를 종합적으로 고려하면 **XGBoost가 더 안정적인 성능**을 보였다고 평가할 수 있습니다.-

---